# Imports

In [ ]:
!pip install rdkit
!pip install lazypredict scikit-learn

In [3]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização de progresso
from tqdm import tqdm

# RDKit para manipulação química
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit import RDLogger

# Pré-processamento
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import NuSVC
from xgboost import XGBClassifier

# Avaliação e validação
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    GridSearchCV,
    StratifiedKFold
)
from sklearn.metrics import make_scorer

# Pipelines
from sklearn.pipeline import Pipeline

# LazyPredict para benchmarking rápido
from lazypredict.Supervised import LazyClassifier

# In vivo

In [8]:
df= pd.read_csv('/content/Dataset in Vivo.csv')
df_estruturas = pd.read_csv('/content/Estruturas de alerta.csv')
df.shape

(11057, 9)

In [10]:
df.drop(columns= ['Chemical','Identificador','Type', 'Species' ,	'Strain' ,	'Male' 	,'Female'],inplace=True)

In [11]:
df['Results'] = df['Results'].str.lower()
df.drop_duplicates(inplace=True)
df.shape

(3611, 2)

In [12]:
# Mapear os valores para 1 (positivos) e 0 (negativos)
mapping = {"positive": 1, "negative": 0, "ambiguous": None, "inconclusive": None}
df["Results"] = df["Results"].map(mapping)

# Remover valores nulos
df = df.dropna()
df.shape

(3611, 2)

In [13]:
# --- Passo 2 ---
# Agora, identificar quais SMILES (no df filtrado) possuem mais de um 'Results'
smiles_mult_result = df.groupby("SMILES")["Results"].nunique()
smiles_mult_result = smiles_mult_result[smiles_mult_result > 1].index

# Remover os SMILES que possuem mais de um resultado
df_final = df[~df["SMILES"].isin(smiles_mult_result)]
df_final.shape

(2223, 2)

In [14]:
df_final["Results"].value_counts()

,count
Results,
0.00,1377
1.00,846


In [15]:
RDLogger.DisableLog('rdApp.*')

def calcular_descritores(smiles):
    """Calcula todos os descritores RDKit para um dado SMILES."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}  # Retorna dicionário vazio se o SMILES for inválido
    return {desc[0]: desc[1](mol) for desc in Descriptors.descList}

def verificar_subestruturas_e_descritores(df, df_estruturas, smiles_col='SMILES', estrutura_smiles_col='SMILES', estrutura_nome_col='Estrutura de Alerta'):
    """
    Para cada molécula no df, verifica a presença de subestruturas e calcula descritores moleculares.

    Parâmetros:
        df: DataFrame contendo uma coluna de SMILES.
        df_estruturas: DataFrame contendo as subestruturas com seus nomes.
        smiles_col: Nome da coluna no df contendo os SMILES das moléculas.
        estrutura_smiles_col: Nome da coluna no df_estruturas contendo os SMILES das subestruturas.
        estrutura_nome_col: Nome da coluna no df_estruturas contendo os nomes das subestruturas.

    Retorna:
        DataFrame df atualizado com colunas de subestruturas (0/1) e descritores moleculares.
    """

    # Converte os padrões do DataFrame df_estruturas em objetos RDKit
    padroes = {
        nome: Chem.MolFromSmarts(smiles) for nome, smiles in zip(df_estruturas[estrutura_nome_col], df_estruturas[estrutura_smiles_col])
        if Chem.MolFromSmarts(smiles) is not None
    }

    # Criar dicionários para armazenar os resultados
    subestrutura_resultados = {nome: [] for nome in padroes}
    descritores_resultados = {desc[0]: [] for desc in Descriptors.descList}

    # Iterar sobre cada SMILES do df e calcular os resultados, usando tqdm para barra de progresso
    for smiles in tqdm(df[smiles_col], desc="Processando moléculas", unit="molécula"):
        mol = Chem.MolFromSmiles(smiles)

        # Verificação de subestruturas
        for nome, padrao in padroes.items():
            subestrutura_resultados[nome].append(int(mol.HasSubstructMatch(padrao)) if mol else 0)

        # Cálculo dos descritores moleculares
        descritores = calcular_descritores(smiles)
        for desc_nome in descritores_resultados:
            descritores_resultados[desc_nome].append(descritores.get(desc_nome, None))

    # Criar DataFrames com os resultados e concatenar com df
    df_subs = pd.DataFrame(subestrutura_resultados)
    df_descs = pd.DataFrame(descritores_resultados)
    df = pd.concat([df.reset_index(drop=True), df_descs, df_subs], axis=1)

    return df

In [16]:
# Exemplo de uso
df_resultado = verificar_subestruturas_e_descritores(df_final, df_estruturas)
df_resultado.shape

Processando moléculas: 100%|██████████| 2223/2223 [00:37<00:00, 58.58molécula/s]


(2223, 377)

In [17]:
# Definir as features (X) e o alvo (y)
X = df_resultado.drop(columns=['SMILES', 'Results'])  # Remove colunas não numéricas
y = df_resultado['Results']

In [ ]:

# Normalizar os dados (opcional, mas recomendado para alguns modelos)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar o StratifiedKFold (10 folds)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Criar dicionário para armazenar os resultados
results = []

# Iterar sobre os folds
for train_idx, test_idx in cv.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inicializar o LazyClassifier
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

    # Treinar e testar os modelos
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    # Salvar os resultados do fold
    results.append(models)

final_results_vivo = (pd.concat(results).groupby(level=0).agg(['mean', 'std']).sort_values(by=('F1 Score', 'mean'), ascending=False)
)

In [19]:
final_results_vivo

Accuracy      Balanced Accuracy      ROC AUC  \
                                  mean  std              mean  std    mean   
Model                                                                        
ExtraTreesClassifier              0.79 0.02              0.76 0.02    0.76   
RandomForestClassifier            0.79 0.01              0.75 0.02    0.75   
XGBClassifier                     0.78 0.01              0.76 0.01    0.76   
LGBMClassifier                    0.78 0.01              0.75 0.01    0.75   
KNeighborsClassifier              0.78 0.01              0.75 0.01    0.75   
NuSVC                             0.78 0.01              0.74 0.02    0.74   
BaggingClassifier                 0.77 0.01              0.73 0.02    0.73   
SVC                               0.77 0.02              0.73 0.02    0.73   
RidgeClassifier                   0.76 0.03              0.74 0.04    0.74   
LogisticRegression                0.76 0.03              0.73 0.03    0.73   
RidgeClassifierCV                 0.76 0.03              0.73 0.04    0.73   
LinearDiscriminantAnalysis        0.75 0.03              0.73 0.03    0.73   
LinearSVC                         0.75 0.04              0.73 0.04    0.73   
DecisionTreeClassifier            0.72 0.01              0.70 0.01    0.70   
LabelPropagation                  0.73 0.02              0.67 0.02    0.67   
LabelSpreading                    0.73 0.02              0.67 0.02    0.67   
PassiveAggressiveClassifier       0.70 0.03              0.69 0.03    0.69   
AdaBoostClassifier                0.71 0.02              0.67 0.02    0.67   
SGDClassifier                     0.70 0.02              0.68 0.02    0.68   
ExtraTreeClassifier               0.70 0.01              0.69 0.01    0.69   
CalibratedClassifierCV            0.73 0.01              0.66 0.01    0.66   
Perceptron                        0.69 0.03              0.67 0.02    0.67   
NearestCentroid                   0.68 0.07              0.67 0.05    0.67   
BernoulliNB                       0.64 0.04              0.64 0.03    0.64   
DummyClassifier                   0.62 0.00              0.50 0.00    0.50   
QuadraticDiscriminantAnalysis     0.48 0.00              0.56 0.01    0.56   
GaussianNB                        0.43 0.02              0.53 0.01    0.53   

                                   F1 Score      Time Taken       
                               std     mean  std       mean  std  
Model                                                             
ExtraTreesClassifier          0.02     0.79 0.02       0.65 0.16  
RandomForestClassifier        0.02     0.78 0.01       1.09 0.11  
XGBClassifier                 0.01     0.78 0.01       1.83 0.88  
LGBMClassifier                0.01     0.77 0.01       1.14 0.02  
KNeighborsClassifier          0.01     0.77 0.01       0.10 0.03  
NuSVC                         0.02     0.77 0.02       0.75 0.14  
BaggingClassifier             0.02     0.76 0.01       1.51 0.11  
SVC                           0.02     0.76 0.02       0.66 0.09  
RidgeClassifier               0.04     0.76 0.03       0.10 0.04  
LogisticRegression            0.03     0.76 0.03       0.18 0.07  
RidgeClassifierCV             0.04     0.76 0.03       0.26 0.05  
LinearDiscriminantAnalysis    0.03     0.75 0.03       0.31 0.14  
LinearSVC                     0.04     0.75 0.04       6.43 0.80  
DecisionTreeClassifier        0.01     0.72 0.01       0.28 0.02  
LabelPropagation              0.02     0.71 0.02       0.32 0.10  
LabelSpreading                0.02     0.71 0.02       0.43 0.18  
PassiveAggressiveClassifier   0.03     0.70 0.03       0.09 0.03  
AdaBoostClassifier            0.02     0.70 0.02       1.22 0.04  
SGDClassifier                 0.02     0.70 0.02       0.21 0.07  
ExtraTreeClassifier           0.01     0.70 0.01       0.05 0.01  
CalibratedClassifierCV        0.01     0.69 0.01      24.82 2.24  
Perceptron                    0.02     0.69 0.02       0.08 0.02  
NearestCentroi

In [20]:
# Validação cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Métricas que queremos avaliar
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

# Dicionário com modelos, pipelines e grids
modelos = {
    "XGBoost": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6],
            "model__learning_rate": [0.01, 0.1]
        }
    },
    "RandomForest": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", RandomForestClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "ExtraTrees": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", ExtraTreesClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "NuSVC": {
        "pipeline": Pipeline([
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler()),
            ("model", NuSVC(probability=True))
        ]),
        "param_grid": {
            "model__nu": [0.25, 0.5, 0.75],
            "model__kernel": ['rbf', 'poly']
        }
    }
}

# Executar GridSearch para cada modelo e armazenar os resultados
resultados_finais = []

for nome_modelo, config in modelos.items():
    print(f"\n🔍 Treinando modelo: {nome_modelo}")

    grid = GridSearchCV(
        estimator=config["pipeline"],
        param_grid=config["param_grid"],
        scoring=scoring,
        refit="f1",
        cv=cv,
        verbose=1,
        n_jobs=-1,
        return_train_score=True
    )

    grid.fit(X, y)

    # Resultados em DataFrame
    df_resultado = pd.DataFrame(grid.cv_results_)
    df_resultado['modelo'] = nome_modelo
    resultados_finais.append(df_resultado)

    # Mostrar melhores parâmetros do modelo atual
    print(f"✅ Melhor F1 ({nome_modelo}): {grid.best_score_:.4f}")
    print(f"Melhores parâmetros: {grid.best_params_}")

# Juntar todos os resultados
df_comparacao = pd.concat(resultados_finais, ignore_index=True)

# Filtrar e ordenar os principais resultados por F1
colunas_mostrar = ['modelo', 'mean_test_accuracy', 'mean_test_precision',
                   'mean_test_recall', 'mean_test_f1', 'mean_test_roc_auc', 'params']

df_resultados_finais = df_comparacao[colunas_mostrar].sort_values(
    by="mean_test_f1", ascending=False)


🔍 Treinando modelo: XGBoost
Fitting 5 folds for each of 8 candidates, totalling 40 fits
✅ Melhor F1 (XGBoost): 0.6986
Melhores parâmetros: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 100}

🔍 Treinando modelo: RandomForest
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Melhor F1 (RandomForest): 0.5759
Melhores parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}

🔍 Treinando modelo: ExtraTrees
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Melhor F1 (ExtraTrees): 0.4762
Melhores parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}

🔍 Treinando modelo: NuSVC
Fitting 5 folds for each of 6 candidates, totalling 30 fits
✅ Melhor F1 (NuSVC): 0.6947
Melhores parâmetros: {'model__kernel': 'rbf', 'model__nu': 0.25}


In [21]:



# Pipeline com scaler + modelo
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ExtraTreesClassifier(random_state=42))
])

# Estratégia de validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Avaliar o modelo usando cross_validate com múltiplas métricas
resultados = cross_validate(
    pipeline,
    X, y,
    cv=cv,
    scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
    return_train_score=True
)

# Mostrar métricas médias
print("🔍 Resultados médios na validação cruzada:")
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    mean = resultados[f'test_{metric}'].mean()
    std = resultados[f'test_{metric}'].std()
    print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")


🔍 Resultados médios na validação cruzada:
accuracy  : 0.8043 ± 0.0171
precision : 0.7980 ± 0.0380
recall    : 0.6538 ± 0.0453
f1        : 0.7172 ± 0.0280
roc_auc   : 0.8523 ± 0.0233


# In vitro

In [22]:
df= pd.read_csv('/content/Dataset in vitro.csv')
df_estruturas = pd.read_csv('/content/Estruturas de alerta.csv')
df.shape

(3325, 4)

In [25]:
df.drop(columns= ['Chemical','Identificador'],inplace=True)
df['Results'] = df['Results'].str.lower()

In [26]:
# Mapear os valores para 1 (positivos) e 0 (negativos)
mapping = {"positive": 1, "negative": 0, "ambiguous": None, "inconclusive": None}
df["Results"] = df["Results"].map(mapping)

# Remover valores nulos
df = df.dropna()
df.shape

(3325, 2)

In [27]:
# --- Passo 2 ---
# Agora, identificar quais SMILES (no df filtrado) possuem mais de um 'Results'
smiles_mult_result = df.groupby("SMILES")["Results"].nunique()
smiles_mult_result = smiles_mult_result[smiles_mult_result > 1].index

# Remover os SMILES que possuem mais de um resultado
df_final = df[~df["SMILES"].isin(smiles_mult_result)]
df_final.shape

(3062, 2)

In [28]:
df_final['Results'].value_counts()

,count
Results,
1.00,1878
0.00,1184


In [29]:
RDLogger.DisableLog('rdApp.*')

def calcular_descritores(smiles):
    """Calcula todos os descritores RDKit para um dado SMILES."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}  # Retorna dicionário vazio se o SMILES for inválido
    return {desc[0]: desc[1](mol) for desc in Descriptors.descList}

def verificar_subestruturas_e_descritores(df, df_estruturas, smiles_col='SMILES', estrutura_smiles_col='SMILES', estrutura_nome_col='Estrutura de Alerta'):
    """
    Para cada molécula no df, verifica a presença de subestruturas e calcula descritores moleculares.

    Parâmetros:
        df: DataFrame contendo uma coluna de SMILES.
        df_estruturas: DataFrame contendo as subestruturas com seus nomes.
        smiles_col: Nome da coluna no df contendo os SMILES das moléculas.
        estrutura_smiles_col: Nome da coluna no df_estruturas contendo os SMILES das subestruturas.
        estrutura_nome_col: Nome da coluna no df_estruturas contendo os nomes das subestruturas.

    Retorna:
        DataFrame df atualizado com colunas de subestruturas (0/1) e descritores moleculares.
    """

    # Converte os padrões do DataFrame df_estruturas em objetos RDKit
    padroes = {
        nome: Chem.MolFromSmarts(smiles) for nome, smiles in zip(df_estruturas[estrutura_nome_col], df_estruturas[estrutura_smiles_col])
        if Chem.MolFromSmarts(smiles) is not None
    }

    # Criar dicionários para armazenar os resultados
    subestrutura_resultados = {nome: [] for nome in padroes}
    descritores_resultados = {desc[0]: [] for desc in Descriptors.descList}

    # Iterar sobre cada SMILES do df e calcular os resultados, usando tqdm para barra de progresso
    for smiles in tqdm(df[smiles_col], desc="Processando moléculas", unit="molécula"):
        mol = Chem.MolFromSmiles(smiles)

        # Verificação de subestruturas
        for nome, padrao in padroes.items():
            subestrutura_resultados[nome].append(int(mol.HasSubstructMatch(padrao)) if mol else 0)

        # Cálculo dos descritores moleculares
        descritores = calcular_descritores(smiles)
        for desc_nome in descritores_resultados:
            descritores_resultados[desc_nome].append(descritores.get(desc_nome, None))

    # Criar DataFrames com os resultados e concatenar com df
    df_subs = pd.DataFrame(subestrutura_resultados)
    df_descs = pd.DataFrame(descritores_resultados)
    df = pd.concat([df.reset_index(drop=True), df_descs, df_subs], axis=1)

    return df

In [ ]:
# Exemplo de uso
df_resultado = verificar_subestruturas_e_descritores(df_final, df_estruturas)
df_resultado.shape

In [33]:
# Definir as features (X) e o alvo (y)
X = df_resultado.drop(columns=['SMILES', 'Results'])  # Remove colunas não numéricas
y = df_resultado['Results']

In [ ]:

# Normalizar os dados (opcional, mas recomendado para alguns modelos)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar o StratifiedKFold (10 folds)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Criar dicionário para armazenar os resultados
results = []

# Iterar sobre os folds
for train_idx, test_idx in cv.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inicializar o LazyClassifier
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

    # Treinar e testar os modelos
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    # Salvar os resultados do fold
    results.append(models)

final_results_vitro = (pd.concat(results).groupby(level=0).agg(['mean', 'std']).sort_values(by=('F1 Score', 'mean'), ascending=False)
)

In [35]:
final_results_vitro

Accuracy      Balanced Accuracy      ROC AUC  \
                                  mean  std              mean  std    mean   
Model                                                                        
ExtraTreesClassifier              0.77 0.01              0.75 0.02    0.75   
RandomForestClassifier            0.75 0.01              0.72 0.02    0.72   
LGBMClassifier                    0.75 0.01              0.72 0.01    0.72   
XGBClassifier                     0.75 0.00              0.72 0.00    0.72   
NuSVC                             0.75 0.02              0.72 0.03    0.72   
KNeighborsClassifier              0.74 0.01              0.71 0.02    0.71   
RidgeClassifierCV                 0.73 0.02              0.71 0.02    0.71   
BaggingClassifier                 0.73 0.00              0.71 0.00    0.71   
SVC                               0.74 0.01              0.70 0.02    0.70   
LinearDiscriminantAnalysis        0.73 0.02              0.70 0.02    0.70   
RidgeClassifier                   0.73 0.01              0.70 0.02    0.70   
LogisticRegression                0.73 0.01              0.70 0.02    0.70   
LinearSVC                         0.72 0.02              0.70 0.02    0.70   
ExtraTreeClassifier               0.69 0.02              0.68 0.03    0.68   
PassiveAggressiveClassifier       0.69 0.01              0.67 0.01    0.67   
DecisionTreeClassifier            0.69 0.01              0.67 0.01    0.67   
LabelSpreading                    0.68 0.02              0.71 0.03    0.71   
LabelPropagation                  0.68 0.02              0.71 0.03    0.71   
SGDClassifier                     0.68 0.01              0.66 0.01    0.66   
Perceptron                        0.68 0.01              0.65 0.00    0.65   
AdaBoostClassifier                0.67 0.02              0.63 0.02    0.63   
CalibratedClassifierCV            0.69 0.01              0.61 0.01    0.61   
NearestCentroid                   0.61 0.01              0.60 0.02    0.60   
BernoulliNB                       0.61 0.01              0.59 0.01    0.59   
QuadraticDiscriminantAnalysis     0.51 0.05              0.54 0.04    0.54   
DummyClassifier                   0.61 0.00              0.50 0.00    0.50   
GaussianNB                        0.48 0.05              0.56 0.03    0.56   

                                   F1 Score      Time Taken       
                               std     mean  std       mean  std  
Model                                                             
ExtraTreesClassifier          0.02     0.76 0.01       0.93 0.28  
RandomForestClassifier        0.02     0.75 0.02       1.35 0.02  
LGBMClassifier                0.01     0.74 0.01       2.38 1.99  
XGBClassifier                 0.00     0.74 0.00       3.87 3.02  
NuSVC                         0.03     0.74 0.03       1.39 0.29  
KNeighborsClassifier          0.02     0.73 0.01       0.15 0.03  
RidgeClassifierCV             0.02     0.73 0.02       0.45 0.15  
BaggingClassifier             0.00     0.73 0.00       2.73 1.49  
SVC                           0.02     0.73 0.02       1.85 0.43  
LinearDiscriminantAnalysis    0.02     0.73 0.02       0.26 0.01  
RidgeClassifier               0.02     0.73 0.01       0.16 0.06  
LogisticRegression            0.02     0.72 0.02       0.25 0.13  
LinearSVC                     0.02     0.72 0.02      10.66 0.77  
ExtraTreeClassifier           0.03     0.69 0.02       0.06 0.00  
PassiveAggressiveClassifier   0.01     0.69 0.01       0.13 0.01  
DecisionTreeClassifier        0.01     0.68 0.01       0.31 0.01  
LabelSpreading                0.03     0.68 0.02       0.62 0.14  
LabelPropagation              0.03     0.68 0.02       0.52 0.13  
SGDClassifier                 0.01     0.67 0.01       0.32 0.08  
Perceptron                    0.00     0.67 0.00       0.09 0.02  
AdaBoostClassifier            0.02     0.66 0.02       1.97 0.66  
CalibratedClassifierCV        0.01     0.64 0.02      41.12 1.50  
NearestCentroi

In [36]:

# Validação cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Métricas que queremos avaliar
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

# Dicionário com modelos, pipelines e grids
modelos = {
    "XGBoost": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6],
            "model__learning_rate": [0.01, 0.1]
        }
    },
    "RandomForest": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", RandomForestClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "ExtraTrees": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", ExtraTreesClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "NuSVC": {
        "pipeline": Pipeline([
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler()),
            ("model", NuSVC(probability=True))
        ]),
        "param_grid": {
            "model__nu": [0.25, 0.5, 0.75],
            "model__kernel": ['rbf', 'poly']
        }
    }
}

# Executar GridSearch para cada modelo e armazenar os resultados
resultados_finais = []

for nome_modelo, config in modelos.items():
    print(f"\n🔍 Treinando modelo: {nome_modelo}")

    grid = GridSearchCV(
        estimator=config["pipeline"],
        param_grid=config["param_grid"],
        scoring=scoring,
        refit="f1",
        cv=cv,
        verbose=1,
        n_jobs=-1,
        return_train_score=True
    )

    grid.fit(X, y)

    # Resultados em DataFrame
    df_resultado = pd.DataFrame(grid.cv_results_)
    df_resultado['modelo'] = nome_modelo
    resultados_finais.append(df_resultado)

    # Mostrar melhores parâmetros do modelo atual
    print(f"✅ Melhor F1 ({nome_modelo}): {grid.best_score_:.4f}")
    print(f"Melhores parâmetros: {grid.best_params_}")

# Juntar todos os resultados
df_comparacao = pd.concat(resultados_finais, ignore_index=True)

# Filtrar e ordenar os principais resultados por F1
colunas_mostrar = ['modelo', 'mean_test_accuracy', 'mean_test_precision',
                   'mean_test_recall', 'mean_test_f1', 'mean_test_roc_auc', 'params']

df_resultados_finais = df_comparacao[colunas_mostrar].sort_values(
    by="mean_test_f1", ascending=False)


🔍 Treinando modelo: XGBoost
Fitting 5 folds for each of 8 candidates, totalling 40 fits
✅ Melhor F1 (XGBoost): 0.8058
Melhores parâmetros: {'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 100}

🔍 Treinando modelo: RandomForest
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Melhor F1 (RandomForest): 0.7914
Melhores parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}

🔍 Treinando modelo: ExtraTrees
Fitting 5 folds for each of 4 candidates, totalling 20 fits
✅ Melhor F1 (ExtraTrees): 0.7803
Melhores parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}

🔍 Treinando modelo: NuSVC
Fitting 5 folds for each of 6 candidates, totalling 30 fits
✅ Melhor F1 (NuSVC): 0.8086
Melhores parâmetros: {'model__kernel': 'rbf', 'model__nu': 0.5}


In [37]:

# Pipeline com scaler + modelo
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", ExtraTreesClassifier(random_state=42))
])

# Estratégia de validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

print("🔍 Treinando modelo: ExtraTreesClassifier")
# Avaliar o modelo usando cross_validate com múltiplas métricas
resultados = cross_validate(
    pipeline,
    X, y,
    cv=cv,
    scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
    return_train_score=True
)

# Mostrar métricas médias formatadas
print("✅ Resultados médios na validação cruzada (10 folds):")
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    mean = resultados[f'test_{metric}'].mean()
    std = resultados[f'test_{metric}'].std()
    print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")


🔍 Treinando modelo: ExtraTreesClassifier
✅ Resultados médios na validação cruzada (10 folds):
accuracy  : 0.7685 ± 0.0142
precision : 0.7998 ± 0.0174
recall    : 0.8312 ± 0.0232
f1        : 0.8149 ± 0.0115
roc_auc   : 0.8376 ± 0.0236


# Conclusão

## *Dados in vivo:*

Os resultados da validação cruzada indicam um desempenho satisfatório do modelo na predição dos dados in vivo. A acurácia média de 80,43% sugere uma taxa elevada de classificação correta, refletindo uma boa capacidade preditiva geral.

A precisão média de 79,80% demonstra que o modelo é eficiente em evitar falsos positivos, o que é particularmente relevante em cenários nos quais classificações incorretas de positivos possam ter consequências críticas. Por outro lado, a revocação média de 65,38% indica que uma parte considerável dos exemplos positivos ainda não está sendo corretamente identificada, o que impacta diretamente o F1-score médio de 0,7172, sinalizando um certo desequilíbrio entre precisão e revocação.

O AUC-ROC médio de 0,8523 evidencia uma excelente capacidade discriminativa entre as classes, reforçando o potencial do modelo para aplicações em contextos onde a distinção entre categorias é essencial. Os desvios padrão relativamente baixos observados nas métricas reforçam a estabilidade do desempenho do modelo ao longo das diferentes iterações da validação cruzada.

Em conjunto, os resultados indicam que o modelo apresenta um desempenho robusto e consistente, com boa capacidade de discriminação e estabilidade, embora haja espaço para melhorias no equilíbrio entre sensibilidade e especificidade, especialmente no contexto de dados in vivo.

## *Dados in vitro:*


A validação cruzada do modelo ExtraTreesClassifier demonstrou um desempenho robusto e consistente. A acurácia média de 76,85% indica uma excelente capacidade preditiva geral, refletindo a eficácia do modelo em classificar corretamente os exemplos em ambas as classes.

Os resultados de precisão (79,98%) e revocação (83,12%) mostram que o modelo é eficiente tanto na identificação correta de instâncias positivas quanto na minimização de falsos negativos. O F1-score médio de 0,8149, que representa o equilíbrio entre precisão e revocação, reforça a capacidade do modelo de lidar bem com a identificação das classes relevantes.

Além disso, o AUC-ROC médio de 0,8376 evidencia uma excelente capacidade discriminativa entre as classes, o que é especialmente relevante em contextos com possível assimetria entre classes. Os baixos desvios padrão observados em todas as métricas indicam estabilidade do desempenho ao longo dos diferentes folds da validação cruzada, reforçando a confiabilidade do modelo.

Esses resultados sugerem que o modelo está bem ajustado ao problema proposto, com desempenho consistente e potencial para aplicações práticas. Estratégias adicionais, como análise de importância de variáveis ou avaliação em conjunto com outras abordagens, podem ser consideradas para futuras iterações e refinamentos.

# Conclusion

## In vivo data:

The cross-validation results indicate a satisfactory performance of the model in predicting in vivo data. The mean accuracy of 80.43% suggests a high overall classification rate, reflecting strong predictive capability.

The mean precision of 79.80% shows that the model effectively avoids false positives, which is particularly important in scenarios where incorrect positive classifications may have critical implications. However, the mean recall of 65.38% indicates that a notable portion of true positive instances remains unidentified, which impacts the mean F1-score of 0.7172, suggesting an imbalance between precision and recall.

The mean AUC-ROC of 0.8523 highlights the model’s excellent discriminative capacity between classes, reinforcing its potential for applications where class distinction is critical. The relatively low standard deviations observed across all metrics indicate consistent behavior throughout the cross-validation folds, supporting the model’s reliability.

Overall, these results demonstrate that the model exhibits robust and stable performance on in vivo data, with strong discrimination capabilities. Nevertheless, there is room for improvement in achieving better balance between sensitivity and specificity, which may be crucial depending on the application domain.

## In vitro data:

Cross-validation of the ExtraTreesClassifier model demonstrated robust and consistent performance. The mean accuracy of 76.85% indicates strong overall predictive capability, reflecting the model’s effectiveness in correctly classifying instances from both classes.

The precision (79.98%) and recall (83.12%) scores highlight the model’s efficiency in correctly identifying positive instances while minimizing false negatives. The mean F1-score of 0.8149, representing the harmonic balance between precision and recall, further reinforces the model's ability to effectively capture the target class.

Moreover, the mean AUC-ROC of 0.8376 indicates excellent discriminative power between classes, which is particularly relevant in scenarios involving potential class imbalance. The low standard deviations observed across all metrics suggest that the model performs consistently across different validation folds, emphasizing its stability and reliability.

These results suggest that the model is well-suited to the task at hand, with consistent performance and potential for practical applications. Additional steps, such as feature importance analysis or ensemble strategies, may be explored in future iterations to further enhance model performance and interpretability.